In [ ]:
!pip install -qU langchain_community wikipedia

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [ ]:
from langchain_community.retrievers import WikipediaRetriever

retriever = WikipediaRetriever()

In [ ]:
propaganda_docs = retriever.invoke("Propaganda Techniques")
fallacy_docs = retriever.invoke("List of Fallacies")

In [ ]:
print(propaganda_docs)

[Document(metadata={'title': 'Propaganda techniques', 'summary': 'Propaganda techniques are methods used in propaganda to convince an audience to believe what the propagandist wants them to believe. Many propaganda techniques are based on socio-psychological research. Many of these same techniques can be classified as logical fallacies or abusive power and control tactics.', 'source': 'https://en.wikipedia.org/wiki/Propaganda_techniques'}, page_content='Propaganda techniques are methods used in propaganda to convince an audience to believe what the propagandist wants them to believe. Many propaganda techniques are based on socio-psychological research. Many of these same techniques can be classified as logical fallacies or abusive power and control tactics.\n\n\n== General character ==\n\n\n=== Definition ===\nIn their book Propaganda and Persuasion, authors Garth S. Jowett and Victoria O\'Donnell define propaganda as the "deliberate, systematic attempt to shape perceptions, manipulate

In [ ]:
!pip install -qU langchain_openai qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.4/266.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 20.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.2 which is incompatible.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.2 which is incompatible.
google-cloud-firestore 2.16.1 requires protob

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

··········


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    """
    You are an expert at evaluating text for propaganda.
    Evaluate the question carefully.
    Use the context as an evaluation criteria.
    Use real time information for additional context.
    Think through your answer step by step.
    Provide your response in the following format: The verdict, the percentage chance, the explanation, bulletpoints of propaganda techniques used.
    Context: {context}
    Question: {question}
    """
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = (
    {"context": retriever | (lambda docs: format_docs(propaganda_docs)), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("Haitians eat pets")

'The verdict: The statement is likely propaganda.\n\nThe percentage chance: 75%\n\nThe explanation: The statement "Haitians eat pets" is a sweeping generalization that lacks nuance and context. While it may be true that in times of severe economic hardship, some individuals in Haiti may resort to eating animals not typically considered food in other cultures, such as pets, this does not represent the practices or cultural norms of the entire population. Statements like this can perpetuate harmful stereotypes and stigmatize a group of people without due consideration of the complex socio-economic factors at play. The statement could be used as propaganda to evoke negative emotions or biases against Haitians, particularly in contexts where there is an agenda to depict them negatively.\n\nBulletpoints of propaganda techniques used:\n- **Stereotyping**: Generalizing a behavior to an entire group without considering individual differences or circumstances.\n- **Emotional appeal**: Using pro

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    """
    You are an expert at evaluating text for propaganda.
    Evaluate the question carefully.
    Use the context as an evaluation criteria.
    Use real time information for additional context.
    Think through your answer step by step.
    Provide your response in the following format: The verdict, the percentage chance, the explanation, bulletpoints of propaganda techniques used.
    Context: {context}
    Question: {question}
    """
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("Haitians eat pets")

'**The verdict**: Propaganda\n\n**The percentage chance**: 95%\n\n**The explanation**: The claim that Haitian immigrants are eating pets in Springfield, Ohio, appears to be a baseless rumor that has been widely debunked. Despite being amplified by prominent figures, including political leaders, there is no credible evidence supporting these claims. The spread of such rumors fits the pattern of propaganda aimed at inciting fear and hostility toward a specific group, in this case, Haitian immigrants. The context suggests that these claims are not only unfounded but also racially charged, contributing to existing racial tensions and resulting in harmful consequences such as bomb threats and increased hostility.\n\n**Bulletpoints of propaganda techniques used**:\n\n- **Disinformation**: Spreading false and unfounded claims without evidence.\n- **Fearmongering**: Inciting fear and panic among the public by suggesting a threat that does not exist.\n- **Scapegoating**: Blaming a minority grou

In [ ]:
chain.invoke("Russia's invasion of Ukrain is justified")

'**The Verdict:** Propaganda\n\n**The Percentage Chance:** 95%\n\n**The Explanation:** The assertion that Russia\'s invasion of Ukraine is justified is highly likely to be propaganda. This statement contradicts the widely accepted view by the international community, which has condemned Russia\'s actions as a violation of Ukraine\'s sovereignty and territorial integrity. The narrative of "justification" often seeks to manipulate public perception and rationalize aggressive actions to serve political or ideological objectives. The context provided on Anatoly Shariy’s situation and views indicates a complex geopolitical environment where narratives can be skewed. Despite Shariy\'s criticism of Ukrainian authorities and acknowledgment of Russian involvement, he condemned the 2022 invasion as aggression, aligning with the broader international stance against the invasion.\n\n**Bulletpoints of Propaganda Techniques Used:**\n- **Justification:** Attempting to rationalize or legitimize Russia

In [ ]:
chain.invoke("Hamas invasion of Israel is justified")

'The verdict: Not justified\n\nThe percentage chance: 80% not justified\n\nThe explanation: The invasion by Hamas into Israeli territory, resulting in the deaths of civilians, is widely condemned by the international community and is considered an act of terrorism by many countries. While Hamas has stated its actions were in response to Israeli occupation and perceived injustices, the methods employed—such as targeting civilians and taking hostages—are not typically considered justified under international law or moral standards. The right to self-defense does not extend to acts that deliberately target non-combatants.\n\nBulletpoints of propaganda techniques used:\n- **Emotional Appeal**: The framing of the invasion as a response to long-standing grievances against Israeli occupation can evoke strong emotions and garner sympathy.\n- **Justification through Historical Context**: By linking the invasion to historical grievances and highlighting ongoing issues such as the blockade and se